# Finesse measurements


First step is to do a full sweep over 20 mu

In [1]:
import PyDAQmx as daq
import numpy as np 
import matplotlib.pyplot as plt
from time import sleep

ni = mynicard

manager.startModule('hardware','scope')

Connected to TEKTRONIX,DPO2014B,C010144,CF:91.1CT FV:v1.52 



0

In [2]:
def sweep_function(SampNum, start, stop, freq):
    f = np.zeros(SampNum)
    period = 1/freq
    t = np.linspace(0,period,SampNum)
    slope = (stop-start) / (period/2)
    for i in range(SampNum):
        if t[i] < 1/2 * period:
            f[i] = slope * t[i] + start
        else:
            f[i] = -slope * t[i] + 2 * stop
    return f


def setup_sweep(SampNum, start_voltage, stop_voltage, freq,RepOfSweep):

    data = sweep_function(SampNum, start_voltage, stop_voltage, freq)

    sweep_task = daq.TaskHandle()
    daq.DAQmxCreateTask('sweep_task', daq.byref(sweep_task))

    daq.DAQmxCreateAOVoltageChan(sweep_task,ni.cavity_channel, "", ni._cavity_voltage_range[0],
                                 ni._cavity_voltage_range[1], daq.DAQmx_Val_Volts,None)
    SampRate= freq * SampNum
#Use internal clock for generate the ramp
    daq.DAQmxCfgSampClkTiming(sweep_task,"",SampRate,daq.DAQmx_Val_Rising,daq.DAQmx_Val_FiniteSamps,RepOfSweep * SampNum)

# Write data to DAQ-card
    daq.DAQmxWriteAnalogF64(sweep_task, SampNum,0,100.0,daq.DAQmx_Val_GroupByChannel,data, None, None)

    return sweep_task

def setup_scope(freq, RepOfSweep):
    
    # set the time_scale such that it fits with the sweep
    Aqusition_lenght = 1/ freq * RepOfSweep
    division = Aqusition_lenght / 10
    scope.set_time_scale(division)
    
    #setup the trigger
    
    

def caviy_sweep(SampNum, start_voltage, stop_voltage, freq, RepOfSweep):
    '''sweep # time back and forth'''
    
    sweep_task = setup_sweep(SampNum, start_voltage, stop_voltage, freq, RepOfSweep)
    setup_scope(freq,RepOfSweep+2)
 
    scope.run_single()
    sleep(0.6)
    daq.DAQmxStartTask(sweep_task)
    

    return sweep_task


In [3]:
sweep_task = caviy_sweep(100000, 0,-0.02, 2, 2)

In [4]:
times, volts = scope.aquire_data()

In [5]:
daq.DAQmxStopTask(sweep_task)
daq.DAQmxClearTask(sweep_task)

0

In [11]:
#volts = volts.reshape(3, int(len(volts)/3))
times = times.reshape(3, int(len(times)/3))

In [8]:
volts[0,:]

array([-1.12, -1.12, -1.12, ...,  2.3 ,  2.28,  2.3 ])

In [ ]:

plt.plot(times,volts)
plt.show()